# [1527. Patients With a Condition](https://leetcode.com/problems/patients-with-a-condition/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Patients

<pre>+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| patient_id   | int     |
| patient_name | varchar |
| conditions   | varchar |
+--------------+---------+</pre>
patient_id is the primary key (column with unique values) for this table.
'conditions' contains 0 or more code separated by spaces.
This table contains information of the patients in the hospital.


Write a solution to find the patient_id, patient_name, and conditions of the patients who have Type I Diabetes. Type I Diabetes always starts with DIAB1 prefix.

Return the result table in any order.

The result format is in the following example.



Example 1:

Input:
Patients table:
<pre>+------------+--------------+--------------+
| patient_id | patient_name | conditions   |
+------------+--------------+--------------+
| 1          | Daniel       | YFEV COUGH   |
| 2          | Alice        |              |
| 3          | Bob          | DIAB100 MYOP |
| 4          | George       | ACNE DIAB100 |
| 5          | Alain        | DIAB201      |
+------------+--------------+--------------+</pre>
Output:
<pre>+------------+--------------+--------------+
| patient_id | patient_name | conditions   |
+------------+--------------+--------------+
| 3          | Bob          | DIAB100 MYOP |
| 4          | George       | ACNE DIAB100 |
+------------+--------------+--------------+</pre>
Explanation: Bob and George both have a condition that starts with DIAB1.

In [1]:
# pandas schema

import pandas as pd

data = [[1, 'Daniel', 'YFEV COUGH'], [2, 'Alice', ''], [3, 'Bob', 'DIAB100 MYOP'], [4, 'George', 'ACNE DIAB100'],
        [5, 'Alain', 'DIAB201']]
patients = pd.DataFrame(data, columns=['patient_id', 'patient_name', 'conditions']).astype(
    {'patient_id': 'int64', 'patient_name': 'object', 'conditions': 'object'})


# to spark dataframe

from pyspark.sql.functions import *
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

patients_df = spark.createDataFrame(patients)
patients_df.show(truncate=False)

+----------+------------+------------+
|patient_id|patient_name|conditions  |
+----------+------------+------------+
|1         |Daniel      |YFEV COUGH  |
|2         |Alice       |            |
|3         |Bob         |DIAB100 MYOP|
|4         |George      |ACNE DIAB100|
|5         |Alain       |DIAB201     |
+----------+------------+------------+



In [3]:
# solving in spark dataframe

patients_df \
    .where(
    (col('conditions').like('DIAB1%')) | (col('conditions').like('% DIAB1%'))
).show()

+----------+------------+------------+
|patient_id|patient_name|  conditions|
+----------+------------+------------+
|         3|         Bob|DIAB100 MYOP|
|         4|      George|ACNE DIAB100|
+----------+------------+------------+



In [4]:
# solving in spark dataframe

patients_df \
    .where(
    (col('conditions').contains(' DIAB1')) | (col('conditions').startswith('DIAB1'))
).show()

+----------+------------+------------+
|patient_id|patient_name|  conditions|
+----------+------------+------------+
|         3|         Bob|DIAB100 MYOP|
|         4|      George|ACNE DIAB100|
+----------+------------+------------+



In [5]:
# solving in spark sql

patients_df.createOrReplaceTempView('patients')

spark.sql('''
select * from patients
where conditions LIKE 'DIAB1%' OR conditions LIKE '% DIAB1%'
''').show()

+----------+------------+------------+
|patient_id|patient_name|  conditions|
+----------+------------+------------+
|         3|         Bob|DIAB100 MYOP|
|         4|      George|ACNE DIAB100|
+----------+------------+------------+



In [6]:
spark.stop()